In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exist-2024-clef/train_data_task1.csv
/kaggle/input/exist-2024-clef/dev_data_task3.csv
/kaggle/input/exist-2024-clef/train_data_task2.csv
/kaggle/input/exist-2024-clef/dev_data_task1.csv
/kaggle/input/exist-2024-clef/train_data_task3.csv
/kaggle/input/exist-2024-clef/EXIT2024_tweet_test.csv
/kaggle/input/exist-2024-clef/dev_data_task2.csv


In [2]:
!wandb offline
!wandb disabled

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
W&B disabled.


In [3]:
!pip install datasets
!pip install transformers

In [4]:
from  IPython.display import clear_output
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
import re
import warnings
warnings.filterwarnings('ignore')

2024-04-27 14:18:32.723464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 14:18:32.723573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 14:18:32.897921: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [23]:
train_df = pd.read_csv('/kaggle/input/exist-2024-clef/train_data_task3.csv')
dev_df = pd.read_csv('/kaggle/input/exist-2024-clef/dev_data_task3.csv')
test_df = pd.read_csv('/kaggle/input/exist-2024-clef/EXIT2024_tweet_test.csv')

In [7]:
import re

def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)

    
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    # print("i am preprocessing")
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@USER", t)
        t = URL_RE.sub("HTTPURL", t)
        tweets.append(t)
    return tweets

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate multi-label encoder
task_encoder = MultiLabelBinarizer()

   
def task_hard_encode(df):
    import numpy as np
    # Create a new DataFrame to store the transformed labels
    transformed_df = df.copy()
    task_encoder.fit(all_task_hard_labels)
    all_labels = []
    for i in range(len(df['hard_label'])):
        # Apply eval to each element of the label column
        all_labels.append(eval(df['hard_label'][i]))
    
    transformed_labels = task_encoder.transform(all_labels)
    # print(pd.DataFrame(transformed_labels))
    # transformed_df["hard_label"] = pd.DataFrame(transformed_labels)
    transformed_labels = pd.Series([x.tolist() for x in transformed_labels])
    transformed_df["hard_label"] = transformed_labels
    return transformed_df

def task_hard_decode(df):
    # Inverse transform the binary-encoded vectors back to original labels
    decoded_df = df.copy()
    decode_labels = np.array(decoded_df["hard_label"].tolist())

    # Inverse transform the label column using the provided task_encoder
    decoded_labels = task_encoder.inverse_transform(decode_labels)
    
    # Assign the decoded labels back to the DataFrame
    decoded_df["hard_labels"] = decoded_labels

    return decoded_df

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments,\
                        Trainer
from datasets import Dataset
import pandas as pd

og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

train1_df = og_train1[["tweet","hard_label"]].dropna().reset_index(drop=True)
dev1_df = og_dev1[["tweet","hard_label"]].dropna().reset_index(drop=True)

all_task_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]]).dropna()
all_task_hard_labels = all_task_hard_labels.apply(lambda x: eval(x)).tolist()
print(all_task_hard_labels[0])

train1_df = task_hard_encode(train1_df)

train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

dev1_df = task_hard_encode(dev1_df)
dev1_df["tweet"] = simple_preprocess(dev1_df["tweet"])

test1_df = og_test1
test1_df = test1_df[["tweet"]]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

['OBJECTIFICATION', 'SEXUAL-VIOLENCE']
train1 (6050, 2)
<bound method NDFrame.head of                                                   tweet          hard_label
0     @USER Ignora al otro, es un capullo.El problem...  [0, 0, 0, 1, 1, 0]
1     @USER Si comicsgate se parece en algo a gamerg...  [0, 0, 1, 0, 0, 0]
2     @USER Lee sobre Gamergate, y como eso ha cambi...  [0, 0, 1, 0, 0, 0]
3     @USER @USER @USER Entonces como así es el merc...  [1, 0, 0, 1, 0, 1]
4     @USER Aaah sí. Andrew Dobson. El que se dedicó...  [0, 0, 1, 0, 0, 0]
...                                                 ...                 ...
6045  idk why y’all bitches think having half your a...  [0, 1, 0, 1, 1, 1]
6046  This has been a part of an experiment with @US...  [0, 0, 0, 1, 0, 0]
6047  "Take me already" "Not yet. You gotta be ready...  [0, 0, 0, 0, 1, 0]
6048    @USER why do you look like a whore? /lh HTTPURL  [0, 1, 0, 1, 1, 1]
6049  ik when mandy says “you look like a whore” i l...  [0, 0, 0, 1, 0, 1]

[

In [11]:
train1_df.head

<bound method NDFrame.head of                                                   tweet          hard_label
0     @USER Ignora al otro, es un capullo.El problem...  [0, 0, 0, 1, 1, 0]
1     @USER Si comicsgate se parece en algo a gamerg...  [0, 0, 1, 0, 0, 0]
2     @USER Lee sobre Gamergate, y como eso ha cambi...  [0, 0, 1, 0, 0, 0]
3     @USER @USER @USER Entonces como así es el merc...  [1, 0, 0, 1, 0, 1]
4     @USER Aaah sí. Andrew Dobson. El que se dedicó...  [0, 0, 1, 0, 0, 0]
...                                                 ...                 ...
6045  idk why y’all bitches think having half your a...  [0, 1, 0, 1, 1, 1]
6046  This has been a part of an experiment with @US...  [0, 0, 0, 1, 0, 0]
6047  "Take me already" "Not yet. You gotta be ready...  [0, 0, 0, 0, 1, 0]
6048    @USER why do you look like a whore? /lh HTTPURL  [0, 1, 0, 1, 1, 1]
6049  ik when mandy says “you look like a whore” i l...  [0, 0, 0, 1, 0, 1]

[6050 rows x 2 columns]>

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [13]:
# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)


tokenizer_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    # accelerator = Accelerator(),
    output_dir="./output/best-model/taks3/hard",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate= 1.4822612607719694e-06, 
    weight_decay= 0.004509822968576173, 
    logging_dir="./logs",
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

In [16]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})


In [17]:
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss
500,0.495900,0.395264
1000,0.391900,0.351362


TrainOutput(global_step=1484, training_loss=0.39393833191247, metrics={'train_runtime': 1749.1749, 'train_samples_per_second': 13.568, 'train_steps_per_second': 0.848, 'total_flos': 5529223505946624.0, 'train_loss': 0.39393833191247, 'epoch': 4.0})

In [18]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score

# Get predictions on the validation set
from scipy.special import expit
val_predictions = trainer.predict(val_dataset)
val_pred_probabilities = expit(val_predictions.predictions)

val_pred_labels = np.where(val_pred_probabilities > 0.5, 1, 0)
val_true_labels = val_dataset["label"]

# Calculate evaluation metrics 
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels,average = "weighted")

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.5038167938931297
Validation F1 Score: 0.544934104994517


In [19]:
# Get predictions on the test set
from scipy.special import expit
test_predictions = trainer.predict(test_dataset)
test_pred_probabilities = expit(test_predictions.predictions) #same as sigmoid
test_pred_labels = np.where(test_pred_probabilities > 0.5, 1, 0)

In [20]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

test_probs_list = create_results(test_pred_probabilities.tolist(), task_encoder.classes_)
test_probs_list[:10]

[{'IDEOLOGICAL-INEQUALITY': 0.0417814701795578,
  'MISOGYNY-NON-SEXUAL-VIOLENCE': 0.05180047079920769,
  'NO': 0.9456539750099182,
  'OBJECTIFICATION': 0.057388219982385635,
  'SEXUAL-VIOLENCE': 0.05672871693968773,
  'STEREOTYPING-DOMINANCE': 0.04743848741054535},
 {'IDEOLOGICAL-INEQUALITY': 0.04704846069216728,
  'MISOGYNY-NON-SEXUAL-VIOLENCE': 0.0583072230219841,
  'NO': 0.9192383289337158,
  'OBJECTIFICATION': 0.06456159055233002,
  'SEXUAL-VIOLENCE': 0.0523686558008194,
  'STEREOTYPING-DOMINANCE': 0.052323393523693085},
 {'IDEOLOGICAL-INEQUALITY': 0.05376865714788437,
  'MISOGYNY-NON-SEXUAL-VIOLENCE': 0.05004945769906044,
  'NO': 0.9238836169242859,
  'OBJECTIFICATION': 0.07103224843740463,
  'SEXUAL-VIOLENCE': 0.05304911360144615,
  'STEREOTYPING-DOMINANCE': 0.04925505071878433},
 {'IDEOLOGICAL-INEQUALITY': 0.5781364440917969,
  'MISOGYNY-NON-SEXUAL-VIOLENCE': 0.20313607156276703,
  'NO': 0.13490647077560425,
  'OBJECTIFICATION': 0.18775604665279388,
  'SEXUAL-VIOLENCE': 0.123676

In [21]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_labels"].tolist()
    print("decoded labels length:", len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = list(decoded_labels[i])
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "./output/task3_NICA-1.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(pd.Series(test_pred_labels.tolist()), columns=["hard_label"]), test_probs_list, og_test1[["id"]],og_test1)

decoded labels length: 2076
output generated as json


# Soft

In [22]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    # print("i am preprocessing")
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@USER", t)
        t = URL_RE.sub("HTTPURL", t)
        tweets.append(t)
    return tweets

from sklearn.preprocessing import MultiLabelBinarizer

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    OBJECTIFICATION,'SEXUAL-VIOLENCE',NO,STEREOTYPING-DOMINANCE,IDEOLOGICAL-INEQUALITY,MISOGYNY-NON-SEXUAL-VIOLENCE
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["OBJECTIFICATION"], logits_dict["SEXUAL-VIOLENCE"],logits_dict["NO"],logits_dict["STEREOTYPING-DOMINANCE"],logits_dict["IDEOLOGICAL-INEQUALITY"], logits_dict["MISOGYNY-NON-SEXUAL-VIOLENCE"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"OBJECTIFICATION": logits_list[0], "SEXUAL-VIOLENCE": logits_list[1],"NO": logits_list[2], "STEREOTYPING-DOMINANCE": logits_list[3],"IDEOLOGICAL-INEQUALITY": logits_list[4], "MISOGYNY-NON-SEXUAL-VIOLENCE": logits_list[5]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset


#load given data
import pandas as pd
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
dev1_df = og_dev1[["tweet","soft_label"]].dropna()


test1_df = og_test1
test1_df = test1_df[["tweet"]]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])


print(train1_df.head)
print(test1_df.head)

<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.33333333333333304, 0.33333333333333304, 0.1...  
1     [0.16666666666666602, 0.0, 0.833333333333333, ...  
2                        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]  
3     [0.16666666666666602, 0

In [25]:
train1_df.info

<bound method DataFrame.info of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.33333333333333304, 0.33333333333333304, 0.1...  
1     [0.16666666666666602, 0.0, 0.833333333333333, ...  
2                        [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]  
3     [0.16666666666666602,

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6,ignore_mismatched_sizes=True)

In [30]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/task3/soft",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate= 8.036768439133452e-06, 
    weight_decay= 8.398333136282262e-05, 
    logging_dir="./logs",
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [31]:
import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})


In [32]:
#with cross entropy loss and label smoothing
def custom_loss_fn(logits, soft_labels):
    probs = torch.sigmoid(logits)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss

# Define the trainer for each fold
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss




In [33]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 17.06 MiB is free. Process 3865 has 14.73 GiB memory in use. Of the allocated memory 14.43 GiB is allocated by PyTorch, and 96.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
from scipy.special import expit

# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_probabilities = expit(val_predictions.predictions)

val_pred_labels = np.where(val_pred_probabilities > 0.5, 1, 0)
val_true_labels = [[1 if value > 0.5 else 0 for value in sublist] for sublist in val_dataset["label"]]

# Calculate evaluation metrics 
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels,average = "weighted")

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

In [ ]:
# Get predictions on the test set
test_predictions1 = trainer.predict(test_dataset)

test_pred_probabilities1 = expit(test_predictions1.predictions) #, axis=1 #same as sigmoid
test_pred_labels1 = np.where(test_pred_probabilities1 > 0.5, 1, 0)

In [ ]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

classes_labels = ["OBJECTIFICATION","SEXUAL-VIOLENCE","NO","STEREOTYPING-DOMINANCE","IDEOLOGICAL-INEQUALITY","MISOGYNY-NON-SEXUAL-VIOLENCE"]
test_probs_list1 = create_results(test_pred_probabilities1.tolist(), classes_labels)
test_probs_list1[:10]

In [ ]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id_EXIST"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_labels"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = list(decoded_labels[i])
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "./output/task3_NICA-2.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(pd.Series(test_pred_labels1.tolist()),columns=["hard_label"]), test_probs_list, og_test1[["id"]],og_test1)